**Date** 24/11/2025
**Author** Jordi Torres
Core code based on work by professor Gaillard. 

**EXERCISE 2** Ayagari + employment status + search effort and ex destruction rate

In [ ]:
#libraries 

using Printf;
using Roots;
using Plots;
using Distributions;
using LambertW;

#wider page display
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
const size_k 	= 1000;    #number of grid points     
const size_m 	= 7;       #number of productivity classes

const size_e =2 ; #added!

const kmin 		= 0.0;
const kmax 		= 1700.0;

betapar 	= 0.96;
const alphapar 	= 0.36;
const deltapar 	= 0.08;
const rhopar 	= 3.0;
const labor 	= 1.0219882;

const epsV    = 1.0e-6
const epsdist = 1.0e-8
const epsK    = 1.0e-6   # for debugging


## grid constants
const scale1 = 1.6;
const grmin = (kmin/scale1)-1.0;
const exponen=log((kmax/scale1)-grmin)/(size_k-1);

const sstates = [exp(-0.600000000000000)	  exp(-0.400000000000000)	  exp(-0.200000000000000)	  exp(0.000000000000000)	   exp(0.200000000000000)	   exp(0.400000000000000)	   exp(0.600000000000000)];

const strans = [	
    0.046746218637144	   0.217937777267117	   0.397822606398702	   0.266386738072197	   0.065169922261456	   0.005754191945237	   0.000182545418147;	
	0.023199661746751	   0.149524091076020	   0.369020347246402	   0.333823905199677	   0.110578117872631	   0.013276146769082	   0.000577730089437;	
	0.010548958644399	   0.093657511915497	   0.312761268311836	   0.382193227897354	   0.171253064028981	   0.027919224002876	   0.001666745199056;	
	0.004387354018187	   0.053538402796357	   0.242163972572887	   0.399820541225137	   0.242163972572887	   0.053538402796357	   0.004387354018187;	
	0.001666745199056	   0.027919224002876	   0.171253064028981	   0.382193227897354	   0.312761268311837	   0.093657511915497	   0.010548958644399;	
	0.000577730089436	   0.013276146769082	   0.110578117872631	   0.333823905199677	   0.369020347246403	   0.149524091076020	   0.023199661746751;	
	0.000182545418147	   0.005754191945237	   0.065169922261456	   0.266386738072197	   0.397822606398702	   0.217937777267117	   0.046746218637144];

#extension:

 b= 0.35
 tau_wage=0.25

const etrans = [
    0.45 0.55
    0.04 0.96
]

emplo = [0.0, 1.0]

In [ ]:
function utility(cons::Float64)
    cons_adj = max(cons, 1.0e-8)      
    return (cons_adj^(1.0-rhopar)) / (1.0 - rhopar)
end


function invmargutil(x::Float64)
    x_adj = max(x, 1.0e-12)     # ensure strictly positive
    return x_adj^(-1.0/rhopar)
end

##add!!!!!

In [ ]:
function inter1d(x1::Float64,y1::Float64,y2::Float64) 
    return ((1.0-(x1))*(y1)+(x1)*(y2))
end

function getwage(rrate::Float64)
    return (1.0-alphapar)*(alphapar/(rrate+deltapar))^(alphapar/(1.0-alphapar));
end

function getlevel(igrid::Int64)
    #careful this function assumes that grid starts at 0
    return ( scale1*(exp(exponen*(igrid))+grmin) );
end

function getgrid(x::Float64)
    #careful because indexes start at 1 in Julia, a +1.0 was added
    return (log((x)/scale1-grmin)/exponen)+1.0;
end


function get_x(EV::Float64, κ::Float64, β)
    x_star=0.0
    if EV>=0 ##assumption on ties???
        x_star= (1/κ)*lambertw(κ^2.0 *β*EV)
    end
end

function get_xi(x_star::Float64, κ::Float64)
    xi= 1- exp(-κ*x_star)
end

In [ ]:
function nderiv(val1::Float64,val2::Float64,val3::Float64,x1::Float64,x2::Float64,x3::Float64) 
    return ((1.0 - (x3 - x2)/(x3 - x1))*((val3 - val2)/(x3-x2)) + ((x3 - x2)/(x3 - x1))*((val2 - val1)/(x2-x1)));
end

In [ ]:
function POLICY_we(valueF::Array{Float64,3},dvalueF::Array{Float64,3},save::Array{Float64,3},wagerate::Float64,rrate::Float64, grid::Array{Float64}, betapar::Float64, tau_wage::Float64, b::Float64)

    aendo  = zeros(Float64, size_k, size_m, size_e);
    xendo = zeros(Float64, size_k, size_m, size_e);
    EVendo = zeros(Float64, size_k, size_m, size_e);  

    iter=0;

    critV=10000.0;

    while critV>epsV

        # we need copy to make a separate object
        valueFnew = copy(valueF);

        # COMPUTE EXPECTED VALUE / DERIVATIVE / ENDOGENEOUS GRID 
      @inbounds for iemplo::Int64 in 1:1:(size_e)  

            @inbounds Threads.@threads  for iprod::Int64 in 1:1:(size_m) #this is the s_t
                @inbounds for igrid::Int64 in 1:1:(size_k) #this is a' or a_t+1
                   if emploi==empl_today    
                    #writing the continuation value
                    tempnext  = 0.0;
                    dtempnext = 0.0;
                        @inbounds for iprodnext::Int64 in 1:1:(size_m) #this is the s_t+1
                            @inbounds for iemplonext::Int64 in 1:1:(size_e) #this is the s_t+1

                                ## next period expected value
                                tempnext  += strans[iprod,iprodnext]*etrans[iemplo,iemplonext]*valueF[igrid,iprodnext, iemplonext];
                                #NOTE: need to adapt ETRANS!!! to be only v, 1-v

                                ## next period derivative 
                                dtempnext += strans[iprod,iprodnext]*etrans[iemplo,iemplonext]*dvalueF[igrid,iprodnext, iemplonext];

                                if dtempnext <= 0 || isnan(dtempnext)
                                    @printf("ERROR: dtempnext = %e at (igrid=%d, iprod=%d, iemplo=%d)\n",
                                    dtempnext, igrid, iprod, iemplo)
                                    error("Bad dtempnext")
                                end

                                



                            end 
                        end
                    
                     #Those unemployed, x!!!!!!!
                    
                    else
                        #compute Δ EV 
                       EV_0 += strans[iprod,iprodnext]*ValueF[igrid,iprodnext, 0]
                       EV_1 += strans[iprod,iprodnext]*ValueF[igrid,iprodnext, 1]
                       ΔEV= EV_1- EV_0

                       x_star=get_x(ΔEV,κ, β) ###This should be a vector of state decisions
                        delta_par= get_xi(x_star,κ)

                        ## next period expected value
                        tempnext  += strans[iprod,iprodnext]*(delta_par)*EV_1 + (1-delta_par)*EV0


                                ## next period derivative 
                        dtempnext += strans[iprod,iprodnext]*etrans[iemplo,iemplonext]*dvalueF[igrid,iprodnext, iemplonext];
                        



                    end




                    # computing endogenous grid using the first order condition
                    aendo[igrid,iprod, iemplo]  = (invmargutil(betapar*dtempnext) + grid[igrid] - wagerate * (emplo[iemplo]+ b*(1-emplo[iemplo]))*(1-tau_wage)* sstates[iprod])/(1.0 + rrate);
                    EVendo[igrid,iprod, iemplo] = tempnext;

                end
            end
       end

        ## aendo[igrid,iprod]  -- endogenous grid of assets 
        ## atilde(a' = 1 , s) , ...,  atilde(a' = a_n , s) 
        
        ## aexo 
        ## grid[1], ... , grid[n]




        @inbounds for iemplo::Int64 in 1:1:(size_e) 

            # RESCALING STEP
            @inbounds Threads.@threads for iprod::Int64 in 1:1:(size_m) #current productivity state
                ilast=1;
                @inbounds for igrid::Int64 in 1:1:(size_k) #current level of assets 
                    
                    # build cohexo for simplicity
                    cohexo = (1.0 + rrate) * grid[igrid] + wagerate * sstates[iprod]*(emplo[iemplo]+ b*(1-emplo[iemplo]))*(1-tau_wage);


                    # case 1: borrowing constrained case
                    if grid[igrid] <= aendo[1,iprod, iemplo]

                        save[igrid,iprod, iemplo]   = grid[1];
                        evfnext = EVendo[1,iprod, iemplo];

                    else ## otherwise I am either in case 2 or case 3

                        ## This is an improvement because we do not need to restart from scratch (due to monotonicity). 
                        ## We start with the previous grid point solution.
                        itest=ilast;  

                        # case 2: extrapolation case
                        if grid[igrid] > aendo[size_k,iprod, iemplo]
                            ## extrapolation so vfweight > 1
                            vfweight = (grid[igrid]-aendo[size_k-1,iprod, iemplo])/(aendo[size_k,iprod, iemplo]-aendo[size_k-1,iprod, iemplo]);
                            igridL   = size_k-1;
                            igridH   = size_k;
                        else

                            #finding interpolating points on cohendo
                            while ((itest<(size_k+1)&&(grid[igrid]>aendo[itest,iprod, iemplo])))
                                itest+=1;
                            end
                            
                            ## some sanity checks
                            if itest>(size_k+1)
                                @printf("itest>(size_k+1)");
                                exit()
                            end

                            # standard interior case
                            if itest==1
                                @printf("itest==1");
                                exit()
                            end

                            vfweight = (grid[igrid]-aendo[itest-1,iprod, iemplo])/(aendo[itest,iprod, iemplo]-aendo[itest-1,iprod, iemplo]);
                            igridL   = itest-1;
                            igridH   = itest;

                        end

                        ## linear interpolation of the value function and saving function using above weights and indexes
                        evfnext = inter1d(vfweight,EVendo[igridL,iprod, iemplo],EVendo[igridH,iprod, iemplo]);
                        save[igrid,iprod, iemplo] = inter1d(vfweight,grid[igridL],grid[igridH]);                    

                        ilast = min(size_k-1,itest);

                    end

                    ## construct the new VF
                    valueF[igrid,iprod, iemplo] = utility(cohexo-save[igrid,iprod, iemplo]) + betapar * evfnext;

                end
            end
        end 










        # computing new derivatives and convergence
        critV=0.0;
        @inbounds for iemplo::Int64 in 1:1:(size_e)
            @inbounds Threads.@threads for iprod::Int64 in 1:1:(size_m)
                @inbounds for igrid::Int64 in 1:1:(size_k)
            
                    # interior deriv
                    if (igrid>=3)
                        dvalueF[igrid-1,iprod, iemplo]=nderiv(valueF[igrid-2,iprod, iemplo],valueF[igrid-1,iprod, iemplo],valueF[igrid,iprod, iemplo],grid[igrid-2],grid[igrid-1],grid[igrid]);
                    end

                    critV=max(critV,abs(valueF[igrid,iprod, iemplo]-valueFnew[igrid,iprod, iemplo]));

                    # left corner
                    dvalueF[1,iprod, iemplo]=(valueF[2,iprod, iemplo]-valueF[1,iprod, iemplo])/(grid[2]-grid[1]);

                    # right corner
                    dvalueF[size_k,iprod, iemplo]=(valueF[size_k,iprod, iemplo]-valueF[size_k-1,iprod, iemplo])/(grid[size_k]-grid[size_k-1]);
        
                end
            end
        end 
        iter+=1;
         #@printf("CNVG %4d\t%20.15f\n",iter,critV);

    end
    

end